In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch.nn.functional as F

import spacy
import numpy as np

import random, math, time


/home/chester/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')
tokenizer_de = get_tokenizer('spacy', language='de_core_news_sm')

In [3]:
def yield_tokens_en(data_iter):
    for data in data_iter:
        yield tokenizer_en(data[1].rstrip('\n'))
        
def yield_tokens_de(data_iter):
    for data in data_iter:
        yield tokenizer_de(data[0].rstrip('\n'))
        
# torch 1.10 에서 __iter__ reset 을 하지 못하는 문제가 생겼었는데, 1.11 에서 warning 이 나옴.. lambda 대신에 partial 을 사용하라고 함 

In [4]:
train, valid, test = Multi30k('./.data', split=('train', 'valid', 'test'), language_pair=('de','en'))
en_vocab = build_vocab_from_iterator(yield_tokens_en(train), min_freq=1, specials=['<unk>', '<pad>', '<sos>', '<eos>'], special_first=True)
de_vocab = build_vocab_from_iterator(yield_tokens_de(train), min_freq=1, specials=['<unk>', '<pad>', '<sos>', '<eos>'], special_first=True)

/home/chester/.local/lib/python3.8/site-packages/torch/utils/data/datapipes/utils/common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
/home/chester/.local/lib/python3.8/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "
/home/chester/.local/lib/python3.8/site-packages/torch/utils/data/datapipes/iter/combining.py:180: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [5]:
en_vocab.set_default_index(en_vocab['<unk>'])
de_vocab.set_default_index(de_vocab['<unk>'])

In [6]:
print(len(en_vocab))
print(len(de_vocab))

10837
19214


In [7]:
PAD_IDX = en_vocab['<pad>']
SOS_IDX = en_vocab['<sos>']
EOS_IDX = en_vocab['<eos>']
UNK_IDX = en_vocab['<unk>']

print(UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX)

0 1 2 3


In [8]:
en_text_pipeline = lambda x : en_vocab(tokenizer_en(x))
de_text_pipeline = lambda x : de_vocab(tokenizer_de(x))

In [9]:
def collate_batch(batch):
    de_list, en_list = [], []
    for data in batch:
        processed_de = torch.tensor( de_text_pipeline(data[0]), dtype=torch.int64)
        processed_en = torch.tensor( en_text_pipeline(data[1]), dtype=torch.int64)
        de_list.append( torch.cat( [torch.tensor([SOS_IDX]), processed_de, torch.tensor([EOS_IDX])], dim=0))
        en_list.append( torch.cat( [torch.tensor([SOS_IDX]), processed_en, torch.tensor([EOS_IDX])], dim=0))
    de_list = pad_sequence( de_list, padding_value=PAD_IDX)
    en_list = pad_sequence( en_list, padding_value=PAD_IDX)
    return de_list, en_list


In [10]:
BATCH_SIZE = 128

train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
valid_loader = DataLoader(valid, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [11]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
DEVICE = device

In [12]:
from torch import Tensor
from torch.nn import Transformer
        

In [13]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])
    

In [14]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [15]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)      

In [16]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [17]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(de_vocab)
print(SRC_VOCAB_SIZE)
TGT_VOCAB_SIZE = len(en_vocab)
print(TGT_VOCAB_SIZE)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
# BATCH_SIZE = 128  ## defined in the previous section


transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)


for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(transformer):,} trainable parameters')

transformer = transformer.to(device)



19214
10837
The model has 33,570,389 trainable parameters


In [18]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [19]:

def train_epoch(model, optimizer):
    model.train()
    losses = 0 
    batch_count = 0 
    
    train = Multi30k('./.data', split=('train'), language_pair=('de','en'))
    train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

    
    for src, tgt in train_loader:
        src = src.to(device)
        tgt = tgt.to(device)
        
        tgt_input = tgt[:-1,:]    # 맨 마지막에 EOS 를 빼내는 건데... # 
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        optimizer.zero_grad()
        
        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        
        optimizer.step()
        losses += loss.item()
        batch_count += 1
        
    return losses / batch_count
        
def evaluate(model):
    model.eval()
    losses = 0
    batch_count = 0 

    for src, tgt in valid_loader:
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
        batch_count += 1

    return losses / batch_count       
        

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 36

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


Epoch: 1, Train loss: 5.343, Val loss: 4.110, Epoch time = 26.825s
Epoch: 2, Train loss: 3.764, Val loss: 3.314, Epoch time = 27.160s
Epoch: 3, Train loss: 3.160, Val loss: 2.892, Epoch time = 27.588s
Epoch: 4, Train loss: 2.769, Val loss: 2.635, Epoch time = 28.151s
Epoch: 5, Train loss: 2.479, Val loss: 2.448, Epoch time = 27.935s
Epoch: 6, Train loss: 2.248, Val loss: 2.302, Epoch time = 27.813s
Epoch: 7, Train loss: 2.059, Val loss: 2.192, Epoch time = 27.535s
Epoch: 8, Train loss: 1.894, Val loss: 2.117, Epoch time = 27.933s
Epoch: 9, Train loss: 1.757, Val loss: 2.057, Epoch time = 27.603s
Epoch: 10, Train loss: 1.630, Val loss: 2.009, Epoch time = 27.840s
Epoch: 11, Train loss: 1.520, Val loss: 1.964, Epoch time = 27.480s
Epoch: 12, Train loss: 1.419, Val loss: 1.957, Epoch time = 27.615s
Epoch: 13, Train loss: 1.334, Val loss: 1.969, Epoch time = 27.914s
Epoch: 14, Train loss: 1.251, Val loss: 1.931, Epoch time = 28.365s
Epoch: 15, Train loss: 1.170, Val loss: 1.909, Epoch time

In [ ]:

# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))
